### Imports

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from neuropy import plotting
from joblib import Parallel, delayed
from neuropy.utils.signal_process import (
    filter_sig,
    hilbert_amplitude_stat,
    hilbertfast,
    FourierSg,
)
from dip import diptst
from signal_process_utils import get_sw_theta_bimodal_params
from neuropy.core import Signal
import subjects
from numpy.polynomial.polynomial import polyfit

### Best theta and delta channels

In [ ]:
# sessions = subjects.sd.ratSday3
sessions = subjects.ripple_sess()


for s, sess in enumerate(tqdm(sessions)):
    maze = sess.paradigm["maze"].flatten()
    aw = sess.brainstates_fine["AW"].duration_slice(10, 500)[:15]
    qw = sess.brainstates_fine["QW"].duration_slice(10, 500)[:15]
    nrem = sess.brainstates_fine["NREM"].duration_slice(250, 500)[:5]
    rem = sess.brainstates_fine["REM"].duration_slice(100, 200)[:3]

    states = aw + qw + nrem + rem

    goodchan_grp = sess.probegroup.get_connected_channels()
    good_channels = np.concatenate(goodchan_grp).astype("int")

    lfps = sess.eegfile.get_frames_within_epochs(states, good_channels)
    bimodal_params = Parallel(n_jobs=8)(
        delayed(get_sw_theta_bimodal_params)(_) for _ in lfps
    )
    bimodal_params = np.array(bimodal_params)

    best_channels = {
        "slow_wave": good_channels[np.argmax(bimodal_params[:, 0])],
        "theta": good_channels[np.argmax(bimodal_params[:, 2])],
    }

    # np.save(sess.filePrefix.with_suffix(".best_channels.npy"), best_channels)

In [ ]:
from neuropy.utils.signal_process import FourierSg

lfp = sess.eegfile.get_frames_within_epochs(states, 189)
signal = Signal(traces=lfp, sampling_rate=1250)
spect = FourierSg(signal, window=2, overlap=1)

delta = spect.get_band_power(1, 30)
bins = np.linspace(delta.min(), delta.max(), 200)
hist_delta = np.histogram(delta, bins)[0]

_, ax = plt.subplots(1, 2)
ax[0].imshow(spect.freq_slice(0, 30).traces, aspect="auto", origin="lower", vmax=0.5)
ax[1].plot(hist_delta)

### Test best channels

In [ ]:
sessions = subjects.nsd.ratUday2

for s, sess in enumerate(sessions):
    chan1 = sess.best_channels.slow_wave
    chan2 = 111
    print(chan1, chan2)

    delta = []
    for chan in [chan1, chan2]:
        signal = sess.eegfile.get_signal(chan)
        freqs = np.geomspace(1, 100, 100)
        # freqs = freqs[(freqs < 57) | (freqs > 63)]
        spect = FourierSg(signal, window=2, overlap=1, freqs=freqs, norm_sig=True)
        delta.append(spect.get_band_power(1, 4))

In [ ]:
_, ax = plt.subplots(2, 1, sharex=True)

for i in range(2):
    x = delta[i]
    max_val = stats.scoreatpercentile(x, 90)
    print(max_val)
    bins = np.linspace(x.min(), max_val, 200)
    hist_x = np.histogram(x, bins)[0]
    # ax[i].plot(bins[:-1], hist_x)
    # ax[i].plot(bins[:-1], hist_x)
    ax[i].plot(spect.time / 60, delta[i])
    # ax[i].set_ylim(-0.1, 3)
    # ax[i].set_ylim(-0.01, top=stats.scoreatpercentile(delta[i], 99.8))

In [ ]:
_, ax = plt.subplots()
# ax.plot(stats.zscore(spect.traces.sum(axis=0)))
ax.plot(spect.get_band_power(1, 4))
ax.plot(spect.get_band_power(9, 15))

In [83]:
wake = (
    (sess.brainstates_fine["QW"] + sess.brainstates_fine["AW"])
    .set_labels("WK")
    .merge_neighbors()
)
rem = sess.brainstates_fine["REM"]

In [6]:
isinstance("sdfd sdf", str)

True